In [ ]:
import os, glob                     # 파일 경로 탐색용
import numpy as np                 # 배열 연산
from PIL import Image              # 이미지 처리용
import matplotlib.pyplot as plt    # 이미지 시각화
from sklearn.model_selection import train_test_split  # 학습/검증 데이터 분리
from tensorflow.python.keras.utils import np_utils    # one-hot 인코딩


In [ ]:
root_dir = "/content/drive/MyDrive/data/강아지분류/dog/"
categories = ["말티즈", "요크셔", "푸들"]
num_class = len(categories)
image_width = 64
image_height = 64


In [ ]:
X = []  # 이미지 저장 리스트
Y = []  # 이미지의 카테고리 인덱스 저장 리스트

for idx, category in enumerate(categories):  # 카테고리 처리
    image_dir = root_dir + category
    files = glob.glob(image_dir + "/*.*")  # 모든 확장자 포함 처리

    for i, f in enumerate(files):  # 각 카테고리의 이미지 파일 처리
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_width, image_height))
        data = np.asarray(img)
        X.append(data)
        Y.append(idx)
        print(f"{f} : {i+1}")  # 파일 경로와 인덱스 출력

    print(f"{image_dir} 폴더에서 {len(files)}개 이미지 처리 완료")

# 작업 확인용 출력
print(f"image shape : {X[0].shape}")  # 첫 이미지의 shape 확인
print("X 데이터 총 개수:", len(X))   # 이미지 파일 수
print("Y 데이터 총 개수:", len(Y))   # 라벨 수


In [ ]:
np_x = np.array(X)
np_y = np.array(Y)
print(np_x.shape)
print(np_y.shape)


In [ ]:
np_x = np.array(X)
np_y = np.array(Y)

print(np_x.shape)
# 작업 확인용 출력 ⇒ (이미지 총 개수, 가로 크기, 세로 크기, 색상 채널 수)

print(np_y.shape)
# 작업 확인용 출력 ⇒ (이미지 총 개수,)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(np_x, np_y, test_size=0.2, random_state=42)


In [ ]:
X_train = X_train.astype("float") / 255
X_test = X_test.astype("float") / 255


In [ ]:
Y_train = np_utils.to_categorical(Y_train, num_class)
Y_test = np_utils.to_categorical(Y_test, num_class)

print(Y_train[0])


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
input_shape = (image_width, image_height, 3)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())                           # 2D → 1D 벡터
model.add(Dense(128, activation='relu'))       # 은닉층
model.add(Dense(num_class, activation='softmax'))  # 출력층 (클래스 수만큼 노드)


In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)


In [ ]:
MODEL_DIR = root_dir + 'model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
modelpath = MODEL_DIR + "{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(
    filepath=modelpath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5
)


In [ ]:
model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=30,
    batch_size=20,
    verbose=0,
    callbacks=[early_stopping_callback, checkpointer]
)


In [ ]:
score = model.evaluate(X_test, Y_test)
print('loss =', score[0])
print('accuracy =', score[1])


In [ ]:
new_files = glob.glob(root_dir + "new/*.*")
new_files

In [ ]:
image_size = 64
new_img = []   # PIL 이미지 객체 저장용
new_X = []     # 정규화된 NumPy 배열 저장용

for i, fname in enumerate(new_files):
    img = Image.open(fname)                # 이미지 열기
    img = img.convert("RGB")               # RGB로 변환
    img = img.resize((image_size, image_size))  # 64x64로 리사이즈
    new_img.append(img)                    # PIL 객체 저장

    new_data = np.asarray(img)             # 배열로 변환
    new_data = new_data.astype("float") / 255  # 정규화
    new_X.append(new_data)                 # 리스트에 추가

new_X = np.array(new_X)                    # 배열로 변환


In [ ]:
new_Predict = model.predict(new_X)


In [ ]:
for i, c in enumerate(new_Predict):
plt.figure(figsize=(1.5, 1.5))
plt.imshow(new_img[i])
plt.axis('off')
plt.show()

top_1 = c.argmax()
if c[top_1] > 0.78:
    print(f"입력:{new_files[i]} => 예측: {categories[top_1]} / Score={c[top_1]*100:.2f}%\n")
else:
    top_2 = np.argsort(c)[-2]
    print(f"입력:{new_files[i]} => 예측: {categories[top_1]} ({c[top_1]*100:.2f}%) 와 {categories[top_2]} ({c[top_2]*100:.2f}%)의 믹스견으로 추정됩니다.\n")
